In [1]:
from urllib.request import urlopen, Request
from urllib.request import urlopen
from matplotlib import pyplot as plt
import math
import datetime
from time import sleep
from bs4 import BeautifulSoup
import copy
import json
import sys
import csv
import pandas as pd
import numpy as np

In [2]:
def InitializeDataframe():
    dff = pd.DataFrame(columns=['date'])
    startDate = "26/03/2019"
    endDate = "11/10/2019"
    startDate = datetime.datetime.strptime(startDate, "%d/%m/%Y")
    endDate = datetime.datetime.strptime(endDate, "%d/%m/%Y")
    insertedDay = startDate
    dates = []
    while insertedDay < endDate:
        dates.append(insertedDay)
        insertedDay = insertedDay + datetime.timedelta(days=1)
    dff.date = dates
    dff = dff.set_index('date')
    return dff

def EncodeDayOfWeek(dff):
    dff['dayOfWeek_sin'] = 0.0
    dff['dayOfWeek_cos'] = 0.0
    dff['is_Weekend'] = 0
    for index, row in dff.iterrows():
        dayOfWeek = index.weekday()
        dff.at[index,'dayOfWeek_sin'] = math.sin((2*math.pi)/7*dayOfWeek)
        dff.at[index,'dayOfWeek_cos'] = math.cos((2*math.pi)/7*dayOfWeek)
        if dayOfWeek == 6 or dayOfWeek == 5:
            dff.at[index,'is_Weekend'] = 1
        else:
            dff.at[index,'is_Weekend'] = 0
    return dff

def CutPdDate(df):
    for index, row in df.iterrows():
        df.at[index, 'date'] = df.at[index, 'date'][:-5]
    return df

def AddDataFromCsv(dff, path, name, cutTime = True):
    transpd = pd.read_csv(path, delimiter=';', names=['date', 'value'])
    transpd.drop(transpd.head(1).index, inplace=True)
    if cutTime == True:
        transpd = CutPdDate(transpd)
    dff[name] = 0
    
    for index1, row1 in dff.iterrows():
        for index2, row2 in transpd.iterrows():
            date = datetime.datetime.strptime(row2['date'], "%d.%m.%Y")
            if(index1 == date):
                dff.at[index1, name] = float(row2['value'])
    diff = np.diff(dff[name])        
    diff = np.insert(diff, 0, [0])
    dff['d_'+name] = diff
    return dff

def AddOilDataFromCsv(dff, path, name, cutTime = True):
    transpd = pd.read_csv(path, delimiter=';', names=['date', 'value'])
    transpd.drop(transpd.head(1).index, inplace=True)
    if cutTime == True:
        transpd = CutPdDate(transpd)
    dff[name] = 0.0
    
    for index1, row1 in dff.iterrows():
        for index2, row2 in transpd.iterrows():
            date = datetime.datetime.strptime(row2['date'], "%d.%m.%Y")
            if(index1 == date):
                dff.at[index1, name] = float(row2['value'])
    
    lastPrice = 0;
    for index2, row2 in dff.iterrows():
        if row2[name] == 0.0:
            dff.at[index2, name] = lastPrice
        lastPrice = dff.at[index2, name]       
                
    diff = np.diff(dff[name])        
    diff = np.insert(diff, 0, [0])
    dff['d_'+name] = diff
    return dff

def AddCurrency(dff, path, name):
    dff[name+'_open_price'] = 0.0
    dff[name+'_close_price'] = 0.0
    dff[name+'_volume'] = 0.0
    dff[name+'_market_cap'] = 0.0
    dff['d_'+name+'_open_price'] = 0.0
    dff['d_'+name+'_close_price'] = 0.0
    dff['d_'+name+'_volume'] = 0.0
    dff['d_'+name+'_market_cap'] = 0.0
     
    transpd = pd.read_csv(path, delimiter=';')
    for index1, row1 in dff.iterrows():
        for index2, row2 in transpd.iterrows():
            date = datetime.datetime.strptime(row2['date'], "%d.%m.%Y")
            if(index1 == date):
                dff.at[index1, name+'_open_price'] = float(row2['open_price_'+name])
                dff.at[index1, name+'_close_price'] = float(row2['close_price_'+name])
                dff.at[index1, name+'_volume'] = float(row2['volume_'+name])
                dff.at[index1, name+'_market_cap'] = float(row2['market_cap_'+name])
                continue
                
    dff['d_'+name+'_open_price'] = np.insert(np.diff(dff[name+'_open_price']), 0, 0)
    dff['d_'+name+'_close_price'] = np.insert(np.diff(dff[name+'_close_price']), 0, 0)
    dff['d_'+name+'_volume'] = np.insert(np.diff(dff[name+'_volume']), 0, 0)
    dff['d_'+name+'_market_cap'] = np.insert(np.diff(dff[name+'_market_cap']), 0, 0)
    return dff
                
    
def AddStockDataFromCsv(dff, path, name, cutTime = True):
    oldname = name
    name = name+'_price'
    transpd = pd.read_csv(path, delimiter=';')
    if cutTime == True:
        transpd = CutPdDate(transpd)
    dff[name] = 0.0
    dff[oldname+'_volume'] = 0.0
    
    for index1, row1 in dff.iterrows():
        for index2, row2 in transpd.iterrows():
            date = datetime.datetime.strptime(row2['date'], "%d.%m.%Y")
            if(index1 == date):
                dff.at[index1, name] = float(row2['close'])
                dff.at[index1, oldname+'_volume'] = float(row2['volume'])             

    lastPrice = 0;
    for index2, row2 in dff.iterrows():
        if row2[name] == 0.0:
            dff.at[index2, name] = lastPrice
        lastPrice = dff.at[index2, name]       
                
    diff = np.diff(dff[name])        
    diff = np.insert(diff, 0, [0])
    dff['d_'+name] = diff
    
    diff = np.diff(dff[oldname+'_volume'])        
    diff = np.insert(diff, 0, [0])
    dff['d_'+oldname+'_volume'] = diff
    return dff

def AddMarketPrices(dff):
    marketdf = pd.read_csv("market_data/bitcoin_market_price.csv", delimiter=';')
    dff['open_price'] = 0.0
    dff['close_price'] = 0.0
    dff['market_volume'] = 0.0
    dff['market_cap'] = 0.0
    for indexf, rowf in dff.iterrows():
        print(str(indexf) +' is actually processing!', end='\r')
        for index, row in marketdf.iterrows():
            date_m = datetime.datetime.strptime(row['Date'], "%d.%m.%Y")
            if(indexf == date_m):
                dff.at[indexf,'open_price'] = row['Open'].replace(',','')
                dff.at[indexf,'close_price'] = row['Close'].replace(',','')
                dff.at[indexf,'market_volume'] = row['Volume'].replace(',','')
                dff.at[indexf,'market_cap'] = row['Market Cap'].replace(',','')
                continue
    #dff['target_open_price'] = np.append(np.diff(dff['open_price']), [0])
    #dff['target_close_price'] = np.append(np.diff(dff['close_price']), [0])
    #dff['target_rise_fall'] = (dff.target_close_price>0.0)
    
    
    #dff['d_open_price'] = np.insert(np.diff(dff['open_price']),0, [0])
    #dff['d_close_price'] = np.insert(np.diff(dff['close_price']),0, [0])
    #dff['d_market_volume'] = np.insert(np.diff(dff['market_volume']),0, [0])
    #dff['d_market_cap'] = np.insert(np.diff(dff['market_cap']),0, [0])
    return dff

def AddTwitterSentimentData(dff):
    dff['tweet_volume'] = 0
    
    dff['nonzero_neutral_tweet']  = 0
    dff['nonzero_positive_tweet'] = 0
    dff['nonzero_negative_tweet'] = 0
    dff['nonzero_compound_tweet'] = 0
    
    dff['sum_positive_mul_follow'] = 0.0
    dff['sum_neutral_mul_follow']  = 0.0
    dff['sum_negative_mul_follow'] = 0.0
    dff['sum_compound_mul_follow'] = 0.0
    
    dff['sum_positive'] = 0.0
    dff['sum_neutral']  = 0.0
    dff['sum_negative'] = 0.0
    dff['sum_compound'] = 0.0
    
    dff['avg_sum_positive'] = 0.0
    dff['avg_sum_neutral']  = 0.0
    dff['avg_sum_negative'] = 0.0
    dff['avg_sum_compound'] = 0.0
    
    dff['avg_sum_nonzero_positive'] = 0.0
    dff['avg_sum_nonzero_neutral']  = 0.0
    dff['avg_sum_nonzero_negative'] = 0.0
    dff['avg_sum_nonzero_compound'] = 0.0
    
    dff['avg_sum_positive_mul_follow'] = 0.0
    dff['avg_sum_neutral_mul_follow']  = 0.0
    dff['avg_sum_negative_mul_follow'] = 0.0
    dff['avg_sum_compound_mul_follow'] = 0.0
    
    dff['avg_sum_nonzero_positive_mul_follow'] = 0.0
    dff['avg_sum_nonzero_neutral_mul_follow']  = 0.0
    dff['avg_sum_nonzero_negative_mul_follow'] = 0.0
    dff['avg_sum_nonzero_compound_mul_follow'] = 0.0
        
    for index, row in dff.iterrows():
        print(str(index) +' is actually processing!', end='\r')
        
        filedate = 'data_done/'+str(index).split(' ')[0]+'.csv'
        tdff = pd.read_csv(filedate, encoding="utf-16", engine='python')
        
        tdff['positive_mult_follow'] = tdff.positive * tdff.user_follower
        tdff['neutral_mult_follow']  = tdff.neutral * tdff.user_follower
        tdff['negative_mult_follow'] = tdff.negative * tdff.user_follower
        tdff['compound_mult_follow'] = tdff['compound'] * tdff.user_follower
        
        volume = len(tdff.index)

        nonZero_neut = (tdff.neutral>0.0).sum()
        nonZero_pos  = (tdff.positive>0.0).sum()
        nonZero_neg  = (tdff.negative>0.0).sum()
        nonZero_comp = (tdff['compound']>0.0).sum()
        
        sum_neut = tdff.neutral.sum()
        sum_pos  = tdff.positive.sum()
        sum_neg  = tdff.negative.sum()
        sum_comp = tdff['compound'].sum()
        
        sum_positive_mul_follow = tdff['positive_mult_follow'].sum()
        sum_neutral_mul_follow  = tdff['neutral_mult_follow'].sum()
        sum_negative_mul_follow = tdff['negative_mult_follow'].sum()
        sum_compound_mul_follow = tdff['compound_mult_follow'].sum()
        
        #fill dataframe
        dff.at[index,'tweet_volume'] = volume
        
        dff.at[index,'nonzero_neutral_tweet']  = nonZero_neut
        dff.at[index,'nonzero_positive_tweet'] = nonZero_pos
        dff.at[index,'nonzero_negative_tweet'] = nonZero_neg
        dff.at[index,'nonzero_compound_tweet'] = nonZero_comp
                
        dff.at[index,'sum_positive_mul_follow'] = sum_positive_mul_follow
        dff.at[index,'sum_neutral_mul_follow']  = sum_neutral_mul_follow
        dff.at[index,'sum_negative_mul_follow'] = sum_negative_mul_follow
        dff.at[index,'sum_compound_mul_follow'] = sum_compound_mul_follow
    
        dff.at[index,'sum_positive'] = sum_pos
        dff.at[index,'sum_neutral']  = sum_neut
        dff.at[index,'sum_negative'] = sum_neg
        dff.at[index,'sum_compound'] = sum_comp
               
        dff.at[index,'avg_sum_positive'] = sum_pos  / volume
        dff.at[index,'avg_sum_neutral']  = sum_neut / volume
        dff.at[index,'avg_sum_negative'] = sum_neg  / volume
        dff.at[index,'avg_sum_compound'] = sum_comp / volume
        
        dff.at[index,'avg_sum_nonzero_positive'] = sum_pos  / nonZero_pos
        dff.at[index,'avg_sum_nonzero_neutral']  = sum_neut / nonZero_neut
        dff.at[index,'avg_sum_nonzero_negative'] = sum_neg  / nonZero_neg
        dff.at[index,'avg_sum_nonzero_compound'] = sum_comp / nonZero_comp
        
        dff.at[index,'avg_sum_positive_mul_follow'] = sum_positive_mul_follow / volume
        dff.at[index,'avg_sum_neutral_mul_follow']  = sum_neutral_mul_follow  / volume
        dff.at[index,'avg_sum_negative_mul_follow'] = sum_negative_mul_follow / volume
        dff.at[index,'avg_sum_compound_mul_follow'] = sum_compound_mul_follow / volume
        
        dff.at[index, 'avg_sum_nonzero_positive_mul_follow'] = sum_positive_mul_follow / nonZero_pos
        dff.at[index, 'avg_sum_nonzero_neutral_mul_follow']  = sum_neutral_mul_follow  / nonZero_neut
        dff.at[index, 'avg_sum_nonzero_negative_mul_follow'] = sum_negative_mul_follow / nonZero_neg
        dff.at[index, 'avg_sum_nonzero_compound_mul_follow'] = sum_compound_mul_follow / nonZero_comp
    
    #dff['d_tweet_volume'] = np.insert(np.diff(dff['tweet_volume']),0, [0])
    return dff

def AddCobaltPrice(dff):
    dff['cobalt_price'] = 0.0
    cdf = pd.read_csv("_stockdata/cobalt_data.csv", delimiter=';')
    name = 'cobalt_price'
    for index1, row1 in dff.iterrows():
        for index2, row2 in cdf.iterrows():
            date = datetime.datetime.strptime(row2['Date'], "%d.%m.%Y")
            if(index1 == date):
                dff.at[index1, name] = float(row2['Price'])
    
    #ensure empty dates - fill last known price
    lastPrice = 0;
    for index2, row2 in dff.iterrows():
        if row2['cobalt_price'] == 0.0:
            dff.at[index2, name] = lastPrice
        lastPrice = dff.at[index2, name]
                
    diff = np.diff(dff[name])        
    diff = np.insert(diff, 0, [0])
    dff['d_'+name] = diff
        
    return dff

def AddNewsAggregatedData(dff):
    ndf = pd.read_csv("twitter_data/news_with_sentiment.csv")
    
    for index1, row1 in dff.iterrows():
        print(str(index1) +' is actually processing!', end='\r')

        numberOfDailyNews = 0
        tags = ''
        cleaned_headertext_sent_neg = 0.0
        cleaned_headertext_sent_neut = 0.0
        cleaned_headertext_sent_pos = 0.0
        cleaned_headertext_sent_comp = 0.0
        headertext_sent_neg = 0.0
        headertext_sent_neut = 0.0
        headertext_sent_pos = 0.0
        headertext_sent_comp = 0.0
        contenttext_sent_neg = 0.0
        contenttext_sent_neut = 0.0
        contenttext_sent_pos = 0.0
        contenttext_sent_comp = 0.0
        cleaned_contenttext_sent_neg = 0.0
        cleaned_contenttext_sent_neut = 0.0
        cleaned_contenttext_sent_pos = 0.0
        cleaned_contenttext_sent_comp = 0.0

        for index2, row2 in ndf.iterrows():
            date = datetime.datetime.strptime(str(row2['date']).split(' ')[0], "%Y-%m-%d")
            if(index1 == date):
                numberOfDailyNews = numberOfDailyNews + 1
                tags = tags + row2['tags']
                cleaned_headertext_sent_neg += row2['cleaned_headertext_sent_neg']
                cleaned_headertext_sent_neut += row2['cleaned_headertext_sent_neut']
                cleaned_headertext_sent_pos += row2['cleaned_headertext_sent_pos']
                cleaned_headertext_sent_comp += row2['cleaned_headertext_sent_comp']
                headertext_sent_neg += row2['headertext_sent_neg']
                headertext_sent_neut += row2['headertext_sent_neut']
                headertext_sent_pos += row2['headertext_sent_pos']
                headertext_sent_comp += row2['headertext_sent_comp']
                contenttext_sent_neg += row2['contenttext_sent_neg']
                contenttext_sent_neut += row2['contenttext_sent_neut']
                contenttext_sent_pos += row2['contenttext_sent_pos']
                contenttext_sent_comp += row2['contenttext_sent_comp']
                cleaned_contenttext_sent_neg += row2['cleaned_contenttext_sent_neg']
                cleaned_contenttext_sent_neut += row2['cleaned_contenttext_sent_neut']
                cleaned_contenttext_sent_pos += row2['cleaned_contenttext_sent_pos']
                cleaned_contenttext_sent_comp += row2['cleaned_contenttext_sent_comp']

        if numberOfDailyNews == 0:
            numberOfDailyNews = 1
            
        dff.at[index1,'sum_clheaders_negative']  = cleaned_headertext_sent_neg
        dff.at[index1,'sum_clheaders_positive']  = cleaned_headertext_sent_pos
        dff.at[index1,'sum_clheaders_neutral']  = cleaned_headertext_sent_neut
        dff.at[index1,'sum_clheaders_compound']  = cleaned_headertext_sent_comp

        dff.at[index1,'avg_clheaders_negative']  = cleaned_headertext_sent_neg  / numberOfDailyNews
        dff.at[index1,'avg_clheaders_positive']  = cleaned_headertext_sent_pos  / numberOfDailyNews
        dff.at[index1,'avg_clheaders_neutral']  = cleaned_headertext_sent_neut  / numberOfDailyNews
        dff.at[index1,'avg_clheaders_compound']  = cleaned_headertext_sent_comp / numberOfDailyNews

        dff.at[index1,'sum_headers_negative']  = headertext_sent_neg
        dff.at[index1,'sum_headers_positive']  = headertext_sent_pos
        dff.at[index1,'sum_headers_neutral']  = headertext_sent_neut
        dff.at[index1,'sum_headers_compound']  = headertext_sent_comp

        dff.at[index1,'avg_headers_negative']  = headertext_sent_neg  / numberOfDailyNews
        dff.at[index1,'avg_headers_positive']  = headertext_sent_pos  / numberOfDailyNews
        dff.at[index1,'avg_headers_neutral']  = headertext_sent_neut  / numberOfDailyNews
        dff.at[index1,'avg_headers_compound']  = headertext_sent_comp / numberOfDailyNews

        dff.at[index1,'sum_clcontents_negative']  = cleaned_contenttext_sent_neg
        dff.at[index1,'sum_clcontents_positive']  = cleaned_contenttext_sent_pos
        dff.at[index1,'sum_clcontents_neutral']  = cleaned_contenttext_sent_neut
        dff.at[index1,'sum_clcontents_compound']  = cleaned_contenttext_sent_comp

        dff.at[index1,'avg_clcontents_negative']  = cleaned_contenttext_sent_neg / numberOfDailyNews
        dff.at[index1,'avg_clcontents_positive']  = cleaned_contenttext_sent_pos / numberOfDailyNews
        dff.at[index1,'avg_clcontents_neutral']  = cleaned_contenttext_sent_neut / numberOfDailyNews
        dff.at[index1,'avg_clcontents_compound']  = cleaned_contenttext_sent_comp / numberOfDailyNews

        dff.at[index1,'sum_contents_negative']  = contenttext_sent_neg
        dff.at[index1,'sum_contents_positive']  = contenttext_sent_pos
        dff.at[index1,'sum_contents_neutral']  = contenttext_sent_neut
        dff.at[index1,'sum_contents_compound']  = contenttext_sent_comp

        dff.at[index1,'avg_contents_negative']  = contenttext_sent_neg / numberOfDailyNews
        dff.at[index1,'avg_contents_positive']  = contenttext_sent_pos / numberOfDailyNews
        dff.at[index1,'avg_contents_neutral']  = contenttext_sent_neut / numberOfDailyNews
        dff.at[index1,'avg_contents_compound']  = contenttext_sent_comp / numberOfDailyNews
        
    return dff

def AddFearIndex(dff):
    
    cdf = pd.read_csv("_fearfactorindex/fear_factor.csv", delimiter=';')
    name = 'fear_factor'
    for index1, row1 in dff.iterrows():
        for index2, row2 in cdf.iterrows():
            date = datetime.datetime.strptime(row2['date'], "%d.%m.%Y")
            if(index1 == date):
                dff.at[index1, name+'_index'] = float(row2['fear_index'])
                dff.at[index1, name+'_class'] = row2['clasification']
               
    diff = np.diff(dff[name+'_index'])        
    diff = np.insert(diff, 0, [0])
    dff['d_'+name] = diff
        
    return dff


In [3]:
dff = InitializeDataframe()

In [4]:
dff = AddMarketPrices(dff)

In [5]:
dff = EncodeDayOfWeek(dff)

In [6]:
dff = AddDataFromCsv(dff, 'bitcoin_blockchain_data/confirmed_transactions.csv', 'num_of_transaction')

In [7]:
dff = AddDataFromCsv(dff, 'bitcoin_blockchain_data/miners_revenue.csv', 'miners_revenue')

In [8]:
dff = AddDataFromCsv(dff, 'bitcoin_blockchain_data/output_volume.csv', 'output_volume')

In [9]:
dff = AddDataFromCsv(dff, 'bitcoin_blockchain_data/total_transactions_fees.csv', 'total_trans_fees')

In [10]:
dff = AddDataFromCsv(dff, 'google_trends/bitcoin_trends.csv', 'bitcoin_trends', cutTime = False)

In [11]:
dff = AddDataFromCsv(dff, 'google_trends/cryptocurrency_trends.csv', 'crypto_trends', cutTime = False)

In [12]:
dff = AddTwitterSentimentData(dff)

In [13]:
dff = AddCobaltPrice(dff)

In [14]:
dff = AddOilDataFromCsv(dff, '_stockdata/oil_prices.csv', 'oil_price', cutTime = False)

In [15]:
dff = AddStockDataFromCsv(dff, '_stockdata/amd_prices.csv', 'amd', cutTime = False)

In [16]:
dff = AddStockDataFromCsv(dff, '_stockdata/intel_prices.csv', 'intel', cutTime = False)

In [17]:
dff = AddStockDataFromCsv(dff, '_stockdata/nvidia_prices.csv', 'nvidia', cutTime = False)

In [18]:
dff = AddNewsAggregatedData(dff)

In [19]:
dff = AddFearIndex(dff)

In [20]:
dff = AddCurrency(dff, 'currency_data/btccash_data.csv', 'btccash')

In [21]:
dff = AddCurrency(dff, 'currency_data/ethereum_data.csv', 'ethereum')

In [22]:
dff = AddCurrency(dff, 'currency_data/litecoin_data.csv', 'litecoin')

In [23]:
dff = AddCurrency(dff, 'currency_data/maker_data.csv', 'maker')

In [24]:
dff = AddCurrency(dff, 'currency_data/monero_data.csv', 'monero')

In [25]:
dff = AddCurrency(dff, 'currency_data/xrp_data.csv', 'xrp')

In [26]:
dff.to_csv("_finalDataset/data_f.csv", encoding='utf-8')